# Hugging Face Workshop

## Part 1 — What is Hugging Face?

* ML/AI open-source Hub, platform and community
* Repository for open:
  * pre-trained models and
  * ML/AI ready datasets
  * demo apps

  for various **tasks**
* Offers libraries for interacting with the Hub

**Tasks** describe the “shape” of each model’s API (inputs and outputs).

<img src="https://drive.google.com/uc?export=view&id=1xDAws-cwDVqPWVYkyaValKv896Hf1J88" alt="diagram" width="400"/>

<p align="lect"><sub><i>Image source: Hugging Face Docs</i></sub></p>

* image classification
  * input image -> vector of one score per class -> argmax
* image segmentation:
  * input image -> per-pixel class score -> per-pixel argmax

<img src="https://drive.google.com/uc?export=view&id=1_LxbwlGyqSPx6laM2LqCLdaLWk_jUkDn" alt="diagram" width="400"/>
<p align="left"><sub><i>Image source: ChatGPT generated</i></sub></p>


In [1]:
from datasets import



## Part 2 — Downloading Datasets
(Code cells using `datasets` library.)


In [ ]:
from datasets import load_dataset


## Part 3 — Downloading Models with Transformers

The 🤗 Transformers library provides the functionality to create and use  models.

The Model Hub contains millions of pretrained models that anyone can download and use. You can also upload your own models to the Hub!


## Part 4 — Running LLMs Locally (Ollama & vLLM)
(Notes and optional code snippets showing usage.)

Why is Hugging Face a Community?

Community Collaboration: As a social platform, it fosters collaboration among AI enthusiasts, researchers, and developers to share knowledge, improve models, and work together on projects.

Tools and Resources: Hugging Face provides tools, demos, documentation, and tutorials to help users learn and experiment with AI models, making it a resource for skill development in the AI field.